In [1]:
# import optum.utils
import pandas as pd
import numpy as np
import time
import pickle
import pyarrow as pa
import pyarrow.parquet as pq

pd.set_option('display.max_columns', 500)
# To execute a cell line by line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Extracting Patient IDs for different Division - Race combinations

In [2]:
with open('div_race_dict.pickle', 'rb') as handle:
    b = pickle.load(handle)

# Extracting Prescription data only for patients using opioid drugs

In [3]:
lu_ndc_opioid = pd.read_csv('/N/u/asakiram/Carbonate/Downloads/lu_ndc_opioid.csv')
lu_ndc_opioid.shape
lu_ndc_opioid.head()

(17426, 18)

,AHFSCLSS,AHFSCLSS_DESC,BRND_NM,DOSAGE_FM_DESC,DRG_STRGTH_DESC,DRG_STRGTH_NBR,DRG_STRGTH_UNIT_DESC,DRG_STRGTH_VOL_NBR,DRG_STRGTH_VOL_UNIT_DESC,GNRC_IND,GNRC_NBR,GNRC_NM,GNRC_SQNC_NBR,NDC,NDC_DRG_ROW_EFF_DT,NDC_DRG_ROW_END_DT,USC_ID,USC_MED_DESC
0,28080800,OPIATE AGONISTS,DARVOCET-N 50,TABLET,50MG-325MG,0.0,NaN,0.0,NaN,1.0,70933,PROPOXYPHENE NAP/ACETAMINOPHEN,4274,2035102,2018-03-29 00:00:00.000000000,1816-03-29 05:56:08.066277376,2212.0,DARVOCET-N 50 50 MG-325 MG TABLET
1,28080800,OPIATE AGONISTS,DARVOCET-N 50,TABLET,50MG-325MG,0.0,NaN,0.0,NaN,1.0,70933,PROPOXYPHENE NAP/ACETAMINOPHEN,4274,2035103,2018-03-29 00:00:00.000000000,1816-03-29 05:56:08.066277376,2212.0,DARVOCET-N 50 50 MG-325 MG TABLET
2,28080800,OPIATE AGONISTS,DARVOCET-N 50,TABLET,50MG-325MG,0.0,NaN,0.0,NaN,1.0,70933,PROPOXYPHENE NAP/ACETAMINOPHEN,4274,2035133,2018-03-29 00:00:00.000000000,1816-03-29 05:56:08.066277376,2212.0,DARVOCET-N 50 50 MG-325 MG TABLET
3,28080800,OPIATE AGONISTS,DARVOCET-N 50,TABLET,50MG-325MG,0.0,NaN,0.0,NaN,1.0,70933,PROPOXYPHENE NAP/ACETAMINOPHEN,4274,2035143,2018-03-29 00:00:00.000000000,1816-03-29 05:56:08.066277376,2212.0,DARVOCET-N 50 50 MG-325 MG TABLET
4,28080800,OPIATE AGONISTS,DARVOCET-N 50,TABLET,50MG-325MG,0.0,NaN,0.0,NaN,1.0,70933,PROPOXYPHENE NAP/ACETAMINOPHEN,4274,2035151,2018-03-29 00:00:00.000000000,1816-03-29 05:56:08.066277376,2212.0,DARVOCET-N 50 50 MG-325 MG TABLET


In [4]:
lu_ndc_opioid.drop_duplicates(inplace = True)
lu_ndc_opioid.shape

(17426, 18)

In [5]:
lu_ndc_opioid_codes = lu_ndc_opioid['AHFSCLSS'].unique()
lu_ndc_opioid_codes

array([28080800, 28100000, 28081200,   280808])

In [6]:
lu_ndc_opioid[lu_ndc_opioid['AHFSCLSS'] == 280808]

,AHFSCLSS,AHFSCLSS_DESC,BRND_NM,DOSAGE_FM_DESC,DRG_STRGTH_DESC,DRG_STRGTH_NBR,DRG_STRGTH_UNIT_DESC,DRG_STRGTH_VOL_NBR,DRG_STRGTH_VOL_UNIT_DESC,GNRC_IND,GNRC_NBR,GNRC_NM,GNRC_SQNC_NBR,NDC,NDC_DRG_ROW_EFF_DT,NDC_DRG_ROW_END_DT,USC_ID,USC_MED_DESC
413,280808,OPIATE AGONISTS,DILAUDID,TABLET,3MG,3.0,MG,0.0,NaN,1.0,16142,HYDROMORPHONE HCL,4111,44102302,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,NaN,NaN
2038,280808,OPIATE AGONISTS,ASPIRIN W/CODEINE,TABLET,15-325MG,0.0,NaN,0.0,NaN,1.0,69911,CODEINE PHOSPHATE/ASPIRIN,4145,349865501,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,NaN,NaN
3013,280808,OPIATE AGONISTS,HYDROCODONE W/ACETAMINOPHEN,CAPSULE,5-500MG,0.0,NaN,0.0,NaN,1.0,70320,HYDROCODONE BIT/ACETAMINOPHEN,4199,536591101,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,2232.0,NaN
3593,280808,OPIATE AGONISTS,CODEINE PHOSPHATE INJECTION,VIAL,30MG/ML,30.0,MG,1.0,ML,2.0,16210,CODEINE PHOSPHATE,4177,641010025,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,NaN,NaN
3594,280808,OPIATE AGONISTS,CODEINE PHOSPHATE INJECTION,VIAL,30MG/ML,30.0,MG,1.0,ML,2.0,16210,CODEINE PHOSPHATE,4177,641010026,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,NaN,NaN
3595,280808,OPIATE AGONISTS,CODEINE PHOSPHATE INJECTION,VIAL,60MG/ML,60.0,MG,1.0,ML,2.0,16211,CODEINE PHOSPHATE,4178,641011025,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,NaN,NaN
3596,280808,OPIATE AGONISTS,CODEINE PHOSPHATE INJECTION,VIAL,60MG/ML,60.0,MG,1.0,ML,2.0,16211,CODEINE PHOSPHATE,4178,641011026,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,NaN,NaN
3748,280808,OPIATE AGONISTS,MARGESIC #3,TABLET,30-650MG,0.0,NaN,0.0,NaN,2.0,70135,CODEINE PHOSPHATE/APAP,4167,682080601,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,2232.0,NaN
4065,280808,OPIATE AGONISTS,HYDROMORPHONE HCL,TABLET,1MG,1.0,MG,0.0,NaN,1.0,16140,HYDROMORPHONE HCL,4109,879071401,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,NaN,NaN
4068,280808,OPIATE AGONISTS,HYDROMORPHONE HCL,TABLET,3MG,3.0,MG,0.0,NaN,1.0,16142,HYDROMORPHONE HCL,4111,879071601,1900-01-01 00:00:00.000000000,1816-03-29 05:56:08.066277376,NaN,NaN


# Extracting Prescription data only for patients using opioid AHFSCLSS codes 

In [7]:
file_path = '/N/project/optum/data/parquet/ses_81_202201/ses_'
years = list(range(2008, 2021, 1))
# years = list(range(2020, 2021, 1))
quarters = ['q1', 'q2', 'q3', 'q4']
file_name = ''
file = ''

In [9]:
# file_name = str(yr) + qtr + '.parquet'
# file_name

In [10]:
# file = file_path + 'r' + file_name
# file

In [11]:
# file

In [13]:
# # rx_temp = pq.ParquetDataset(file, use_legacy_dataset = False, filters=[('AHFSCLSS','=',ndc_code)])
# # print("yes")
# rx_temp1 = rx_temp.read().to_pandas()
# print("yes")
# rx_df = rx_df.append(rx_temp1)

In [14]:
start_time = time.time()

rx_df = pd.DataFrame()
for ndc_code in lu_ndc_opioid_codes:
    for yr in years:
        print("Year : ", yr)
        for qtr in quarters:
            print("Quarter : ", qtr)
            file_name = str(yr) + qtr + '.parquet'
            file = file_path + 'r' + file_name
            print("File : ", file)
            try:
                rx_temp = pq.ParquetDataset(file, use_legacy_dataset = False, filters=[('AHFSCLSS','=',ndc_code)])
                print("yes")
                rx_temp1 = rx_temp.read().to_pandas()
                print("yes")
                rx_df = rx_df.append(rx_temp1)
            except FileNotFoundError:
                print("File doesn't exist")
            else:
                break
                
print("--- %s seconds ---" % (time.time() - start_time))

Year :  2008
Quarter :  q1
File :  /N/project/optum/data/parquet/ses_81_202201/ses_r2008q1.parquet
yes


ArrowNotImplementedError: Function 'equal' has no kernel matching input types (array[string], scalar[int64])